In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import nltk
from  collections import Counter
from sklearn.model_selection import train_test_split
from gensim.models.keyedvectors import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from tqdm import tqdm
from collections import Counter
import numpy as np
import contractions
import re
import random
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
stopwords = set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to
[nltk_data]     /data1/home/armangupta/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /data1/home/armangupta/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
input_file = '/data1/home/armangupta/glove.6B/glove.6B.300d.txt'
output_file = '/data1/home/armangupta/Semester 3/DLNLP/Assignment3/gensim_glove.6B.300d.txt'
EMB_SIZE  = 300
gensim_model = KeyedVectors.load_word2vec_format(output_file,binary= False)
word2int = gensim_model.key_to_index

In [3]:
trainfilepath = 'data/Imdb/Train.csv'
train = pd.read_csv(trainfilepath)
train_reviews = train['review'].to_numpy()
label = train['sentiment'].apply(lambda x : 1 if x=='positive' else 0).to_numpy()

In [4]:
train_reviews , val_reviews , label , val_label = train_test_split(train_reviews , label, test_size = 0.1)
print(train_reviews.shape , val_reviews.shape, label.shape, val_label.shape)

(36000,) (4000,) (36000,) (4000,)


In [5]:
test = pd.read_csv('/data1/home/armangupta/Semester 3/DLNLP/Assignment 2/E0334 Assignment2 Test Dataset.csv', names = ['review','sentiment'])
test_reviews = test['review'].to_numpy()
test_label = test['sentiment'].apply(lambda x : 1 if x=='positive' else 0).to_numpy()


In [7]:
class CleanText:
    def __init__(self):
        pass

    def rm_link(self,text):
        return re.sub(r'https?://\S+|www\.\S+', '', text)
        
    def rm_html(self,text):
        return re.sub(r'<[^>]+>', '', text)

    def space_bt_punct(self,text):
        pattern = r'([.,!?-])'
        s = re.sub(pattern, r'\1 ', text)     # add whitespaces between punctuation
        s = re.sub(r'\s{2,}', ' ', s)        # remove double whitespaces    
        return s
    def space_bt_punct2(self,text):
        pattern = r'([.,!?-])'
        s = re.sub(pattern, r' \1 ', text)     # add whitespaces between punctuation
        s = re.sub(r'\s{2,}', ' ', s)        # remove double whitespaces    
        return s
        
    def rm_punct2(self,text):
        return re.sub(r'[\"\#\$\%\&\-\'\(\)\*\+\/\:\;\<\=\>\@\[\\\]\^\_\`\{\|\}\~\.\,\!\?]', ' ', text)

    def rm_number(self,text):
        return re.sub(r'\d+', '', text)

    def rm_whitespaces(self,text):
        return re.sub(r' +', ' ', text)

    def rm_nonascii(self,text):
        return re.sub(r'[^\x00-\x7f]', r'', text)

    def rm_emoji(self,text):
        emojis = re.compile(
            '['
            u'\U0001F600-\U0001F64F'  # emoticons
            u'\U0001F300-\U0001F5FF'  # symbols & pictographs
            u'\U0001F680-\U0001F6FF'  # transport & map symbols
            u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
            u'\U00002702-\U000027B0'
            u'\U000024C2-\U0001F251'
            ']+',
            flags=re.UNICODE
        )
        return emojis.sub(r'', text)

    def spell_correction(self,text):
        return re.sub(r'(.)\1+', r'\1\1', text)

    def contraction(self,text):
        expanded_words = []   
        for word in text.split():
            # using contractions.fix to expand the shortened words
            expanded_words.append(contractions.fix(word))  
        return ' '.join(expanded_words)

    def clean_pipeline_phase1(self,text):
        contract_text = self.contraction(text)
        spell_corrected = self.spell_correction(contract_text)   
        no_link = self.rm_link(contract_text)
        no_html = self.rm_html(no_link)
        space_punct = self.space_bt_punct(no_html)
        return space_punct

    def clean_pipeline_phase2(self,text):
        space_punct = self.space_bt_punct2(text)
        no_punct = self.rm_punct2(space_punct)
        no_number = self.rm_number(no_punct)
        no_whitespaces = self.rm_whitespaces(no_number)
        no_nonasci = self.rm_nonascii(no_whitespaces)
        no_emoji = self.rm_emoji(no_nonasci)
        spell_corrected = self.spell_correction(no_emoji)
        return spell_corrected.strip().lower()


In [8]:
clean = CleanText()


In [9]:
document_text_reviews = [nltk.sent_tokenize(clean.clean_pipeline_phase1(text)) for text in train_reviews]

In [10]:
sent_text_reviews = []
words = []
for document in document_text_reviews:
    tmp = []
    for sent in document:
        t = clean.clean_pipeline_phase2(sent)
        t = t.lower()
        if t != '':
            tmp.append(t)
            words.extend(word_tokenize(t))
    sent_text_reviews.append(tmp)

In [11]:
temp_len = []
for document in document_text_reviews:
    temp_len.append(len(document))

In [12]:
q_80 = (int)(np.quantile(temp_len, 0.8))
q_80

18

In [13]:
random.choices(sent_text_reviews[0] , k=3)

['this movie pretty much sucked',
 'it will be a better movie also if your not in the military',
 'it will be a better movie also if your not in the military']

In [14]:
opt_sent_text_reviews , opt_label = [],[]
for i in range(len(label)):
    document,l = sent_text_reviews[i] , label[i]
    if len(document) > q_80:
        times =(int)( np.round_(len(document)/q_80))
        for j in range(times):
            opt_sent_text_reviews.append(random.sample(document , k= q_80))
            opt_label.append(l)
    else:
        opt_sent_text_reviews.append(document)
        opt_label.append(l)
assert len(opt_sent_text_reviews) == len(opt_label)

In [15]:
temp_len2 = []
for document in opt_sent_text_reviews:
    temp_len2.append(len(document))
np.quantile(temp_len2,[0.8,.9,.95,.99,1])

array([18., 18., 18., 18., 18.])

In [16]:
# ## Intergrating the vocabulary from the Given Data
# corpus = ' '.join(train_reviews)
# words = corpus.split()
counter = Counter(words)
vocab = sorted(counter, key=counter.get, reverse=True)
remain_train_words = set(vocab).difference(word2int.keys())
random_vector_initialisation = np.random.normal(0,3 , size = (len(remain_train_words) , EMB_SIZE))

In [17]:
start = max(list(word2int.values())) + 1
remain_train_word_w2i = dict(zip(remain_train_words,np.arange(start , start+len(remain_train_words))))
word2int.update(remain_train_word_w2i)

In [18]:
word2int['the'] = max(list(word2int.values())) + 1
word2int['<PAD>'] = 0

In [19]:
embedding_vectors = gensim_model.vectors
embeddings = np.vstack((embedding_vectors,random_vector_initialisation))
embeddings = np.vstack((np.zeros((1,EMB_SIZE)) ,embeddings))

In [20]:
sent_train_encoder = [[[word2int[w] for w in  sent.split()] for sent in review]  for review in opt_sent_text_reviews]

In [21]:
max_len_sent = 0
max_len_doc = 0
for review in sent_train_encoder:
    for sent in review:
        max_len_sent = max(len(sent),max_len_sent)
    max_len_doc = max(max_len_doc , len(review))

In [22]:
max_len_doc , max_len_sent

(18, 350)

In [23]:
def padding_sentence(review, pad_len_document =18 ,pad_len_sent = 50 ,pad_id = 0):
    features = np.full((pad_len_document, pad_len_sent) , pad_id , dtype=int )
    for i, sent in enumerate(review):
        if i < pad_len_document:
            features[i, :len(sent)] = np.array(sent)[:pad_len_sent]
    return features

train_features = np.array([padding_sentence(review) for review in sent_train_encoder])

In [24]:
train_features_sent_len = np.sum(train_features != 0 ,axis = 2)
train_features_sent_len

array([[ 5, 14, 28, ...,  0,  0,  0],
       [29, 22, 10, ...,  0,  0,  0],
       [ 4, 17, 11, ...,  0,  0,  0],
       ...,
       [39, 29, 14, ...,  0,  0,  0],
       [20, 15, 26, ...,  0,  0,  0],
       [25, 32, 16, ...,  0,  0,  0]])

In [25]:
train_features_doc_len = np.sum(train_features_sent_len!=0 , axis = 1)
train_features_doc_len.shape

(39137,)

In [26]:
traindataset = TensorDataset(torch.from_numpy(train_features) , torch.from_numpy(np.array(opt_label)) , torch.from_numpy(train_features_sent_len) , torch.from_numpy(train_features_doc_len))
trainloader = DataLoader(traindataset , batch_size= 16 , shuffle=True)


In [27]:
## Val 
document_val_reviews = [nltk.sent_tokenize(clean.clean_pipeline_phase1(text)) for text in val_reviews]
sent_val_reviews = []
for document in document_val_reviews:
    tmp = []
    for sent in document:
        t = clean.clean_pipeline_phase2(sent)
        t = t.lower()
        if t != '':
            tmp.append(t)
    sent_val_reviews.append(tmp)
sent_val_encoder = [[[word2int[w] for w in  sent.split() if w in word2int] for sent in review]  for review in sent_val_reviews]
val_features = np.array([padding_sentence(review) for review in sent_val_encoder])
val_features_sent_len = np.sum(val_features != 0 ,axis = 2)
val_features_doc_len = np.sum(val_features_sent_len!=0 , axis = 1)
valdataset = TensorDataset(torch.from_numpy(val_features) , torch.from_numpy(val_label) , torch.from_numpy(val_features_sent_len) , torch.from_numpy(val_features_doc_len))
valloader = DataLoader(valdataset , batch_size = 32, shuffle =False )

In [28]:
## Test 
document_test_reviews = [nltk.sent_tokenize(clean.clean_pipeline_phase1(text)) for text in test_reviews]
sent_test_reviews = []
words = []
for document in document_test_reviews:
    tmp = []
    for sent in document:
        t = clean.clean_pipeline_phase2(sent)
        t = t.lower()
        if t != '':
            tmp.append(t)
            words.extend(word_tokenize(t))
    sent_test_reviews.append(tmp)
sent_test_encoder = [[[word2int[w] for w in  sent.split() if w in word2int] for sent in review]  for review in sent_test_reviews]
test_features = np.array([padding_sentence(review) for review in sent_test_encoder])
test_features_sent_len = np.sum(test_features != 0 ,axis = 2)
test_features_doc_len = np.sum(test_features_sent_len!=0 , axis = 1)
testdataset = TensorDataset(torch.from_numpy(test_features) , torch.from_numpy(test_label) , torch.from_numpy(test_features_sent_len) , torch.from_numpy(test_features_doc_len))
testloader = DataLoader(testdataset , batch_size = 32, shuffle =False )

In [29]:


def masked_softmax(X ,valid_len):
    if valid_len == None:
      return nn.Softmax(dim = -1)(X)
    maxlen = X.shape[-1]
    repeatlen = X.shape[-2]
    if valid_len.dim() == 1:
        valid_len = valid_len.unsqueeze(1)
    # print(valid_len.shape , valid_len.dim())
    mask = torch.arange(maxlen)[None, :] < valid_len[:,:,None]
    mask = mask.repeat(1,repeatlen,1)
    assert mask.shape == X.shape
    X[~mask] = 1e-32
    # print('X after masking contains nan:' , torch.isnan(X).any() )
    return nn.Softmax(dim = -1)(X)

class MLPAttention(nn.Module):
    def __init__(self, units, k_dim ,q_dim ,dropout):
        super(MLPAttention , self).__init__()
        self.W_k = nn.Linear(k_dim , units ,bias= False)
        self.W_q = nn.Linear(q_dim , units ,bias= False)
        self.v = nn.Linear(units , 1,bias = False)
        self.dropout = nn.Dropout(dropout)

    def forward(self,query ,key ,value,valid_len):
        query, key = self.W_q(query) , self.W_k(key)
        # print('Query ',query.shape , key.shape , value.shape)
        features = query.unsqueeze(dim=2) + key.unsqueeze(dim=1)
        features = torch.tanh(features)
        # print('features : ', features.shape)
        scores = self.v(features).squeeze(dim=-1)
        # print('Scores contain Nan ? : ', torch.isnan(scores).any())
        attention_weights = self.dropout(masked_softmax(scores, valid_len))
        # print('Attn_weigths contain Nan ? : ', torch.isnan(attention_weights).any())
        return torch.bmm(attention_weights , value)
        
class WordEncoder(nn.Module):
    def __init__(self ,embeddings, vocab_size , emb_size, hidden_size , num_layers = 1 ,pad_id = 0):
        super(WordEncoder, self).__init__()
        self.vocab_size = vocab_size
        self.emb_size = emb_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding.from_pretrained(embeddings , freeze = False ,padding_idx= pad_id )
        self.GRU = nn.GRU(input_size = self.emb_size , hidden_size = self.hidden_size , num_layers = self.num_layers , bidirectional = True , batch_first = True)
        # self.linear_relu1 = nn.Sequential(
        #     nn.Linear(2 * self.hidden_size , self.hidden_size*2),
        #     nn.Dropout(0.4),
        #     nn.ReLU()
        # )
        self.linear_relu2 = nn.Sequential(
            nn.Linear(2 * self.hidden_size , self.hidden_size),
            nn.Dropout(0.1),
            nn.ReLU()
        )

    def forward(self, x):
        # h0 = self.initialize_weights(x.shape[0])
        embs = self.embedding(x).float()
        output , hn = self.GRU(embs)
        # output = self.linear_relu1(output)
        return self.linear_relu2(output) , hn
    
    def initialize_weights(self, batch_size):
        h0 = torch.rand(size = (self.num_layers * 2 ,batch_size, self.hidden_size))
        return h0


In [30]:
class HeirachicalAttentionModel(nn.Module):
    def __init__(self ,embeddings , word2int):
        super(HeirachicalAttentionModel , self).__init__()
        self.wordEncoder = WordEncoder(embeddings,vocab_size  = len(word2int) , emb_size= 300 , hidden_size= 300)
        self.attn_sent = MLPAttention(units = 300 , k_dim = 300 , q_dim= 300, dropout = 0.1)
        self.attn_doc = MLPAttention(units = 300 , k_dim = 300 , q_dim= 300, dropout = 0.1)
        self.GRU = nn.GRU(input_size = 300 , hidden_size = 300 , bidirectional = True , batch_first = True)
        self.linear_relu1 = nn.Sequential(
            nn.Linear(300,150),
            nn.Dropout(0.1),
            nn.ReLU()
        )
        # self.linear_relu2 = nn.Sequential(
        #     nn.Linear(150,100),
        #     nn.Dropout(0.4),
        #     nn.ReLU()
        # )
        self.linear_softmax = nn.Sequential(
            nn.Linear(150 , 2 ),
            nn.Softmax(dim = -1)
        )
    
    def forward(self, X , X_sent_len , X_doc_len):
        sent_reps = []
        for i in range(len(X)):
            doc = X[i]
            output , hn = self.wordEncoder(doc)

            context = self.attn_sent(output , output, output , X_sent_len[i])
            sent_rep = torch.mean(context, dim =1)
            # print(torch.isnan(sent_rep).any())
            sent_reps.append(sent_rep.unsqueeze(dim =0))

        sent_reps = torch.concat(sent_reps , dim = 0)
        sent_rep, hn = self.GRU(sent_rep)
        context = self.attn_doc(sent_reps , sent_reps, sent_reps , X_doc_len)
        doc_reps = torch.mean(context, dim =1)
        x1 = self.linear_relu1(doc_reps)
        # x2 = self.linear_relu2(x1)
        out = self.linear_softmax(x1)
        return out

In [35]:
device = 'cuda:3' if torch.cuda.is_available() else 'cpu'
print(f'Using device : {device}')

Using device : cuda:3


In [36]:
lr = 1e-4
model = HeirachicalAttentionModel(torch.from_numpy(embeddings),word2int).to(device)
criteria = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters() , lr = lr)

In [37]:
def train(trainloader ,valloader ,testloader, model, criteria , optimizer ,epochs ,device):

    for epoch in range(epochs):
        train_loss = []
        total  = 0
        acc = 0
        model = model.to(device)
        for input, label , input_sent_len , input_doc_len in tqdm(trainloader):
            optimizer.zero_grad()
            input , label = input.to(device) , label.to(device)
            output = model(input, input_sent_len ,input_doc_len)
            onehotenc_label = nn.functional.one_hot(label,num_classes = 2).float()
            loss = criteria(output, onehotenc_label)
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
            res = torch.argmax(output , dim = 1)
            acc +=torch.sum(res == label)
            total+=len(label)
            del input , label ,output

        val_acc = 0
        val_total = 0
        val_loss = []
        with torch.no_grad():
            for input, label , input_sent_len , input_doc_len in tqdm(valloader):
                input , label = input.to(device) , label.to(device)
                output = model(input, input_sent_len ,input_doc_len)
                onehotenc_label = nn.functional.one_hot(label,num_classes = 2).float()
                loss = criteria(output, onehotenc_label)
                train_loss.append(loss.item())
                res = torch.argmax(output , dim = 1)
                val_acc +=torch.sum(res == label)
                val_total+=len(label)
                val_loss.append(loss.item())
                del input, label , output
            test_acc = 0
            test_total = 0
            test_loss = []
            for input, label , input_sent_len , input_doc_len in tqdm(testloader):
                input , label = input.to(device) , label.to(device)
                output = model(input, input_sent_len ,input_doc_len)
                onehotenc_label = nn.functional.one_hot(label,num_classes = 2).float()
                loss = criteria(output, onehotenc_label)
                train_loss.append(loss.item())
                res = torch.argmax(output , dim = 1)
                test_acc +=torch.sum(res == label)
                test_total+=len(label)
                test_loss.append(loss.item())
                del input,label , output
            model = model.to('cpu')
        print(f'Epoch {epoch} || train loss : {np.mean(train_loss)} || train accuracy : {acc/total} ||\
             val loss : {np.mean(val_loss)} || val accuracy : {val_acc / val_total} ||\
             test loss : {np.mean(test_loss)} || test accuracy : {test_acc / test_total}')

    

In [38]:
train(trainloader ,valloader , testloader,model,  criteria, optimizer , 20 , device)

100%|██████████| 313/313 [00:35<00:00,  8.88it/s]


Epoch 0 || train loss : 0.5244646837950993 || train accuracy : 0.7490609884262085 ||             val loss : 0.449000453710556 || val accuracy : 0.8575000166893005 ||             test loss : 0.4583216201002225 || test accuracy : 0.8471999764442444


100%|██████████| 313/313 [00:34<00:00,  9.10it/s]


Epoch 1 || train loss : 0.42761961206199806 || train accuracy : 0.8819786310195923 ||             val loss : 0.42512209486961366 || val accuracy : 0.8817500472068787 ||             test loss : 0.43446030508215056 || test accuracy : 0.8727999925613403


100%|██████████| 313/313 [00:35<00:00,  8.90it/s]


...

100%|██████████| 313/313 [00:43<00:00,  7.19it/s]


Epoch 13 || train loss : 0.3486521302104203 || train accuracy : 0.9775915145874023 ||             val loss : 0.421666695356369 || val accuracy : 0.8892500400543213 ||             test loss : 0.4218331399245765 || test accuracy : 0.8883999586105347


100%|██████████| 313/313 [00:42<00:00,  7.36it/s]


Epoch 14 || train loss : 0.34825968028561083 || train accuracy : 0.9780769944190979 ||             val loss : 0.41982651567459106 || val accuracy : 0.8907500505447388 ||             test loss : 0.42448814122821577 || test accuracy : 0.885699987411499


100%|██████████| 313/313 [00:40<00:00,  7.71it/s]
